In [2]:
# Importing libraries 

import pandas as pd
import locale
import requests
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import json
from math import sin, cos, sqrt, atan2, radians
from sklearn.cluster import KMeans
import matplotlib.path as mpltPath

locale.setlocale(locale.LC_ALL, 'en_US.UTF8')
pd.set_option('display.max_column',None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize  


print ('Libraries are imported... :)')

Libraries are imported... :)


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:18: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [3]:
# Making the raw table

source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe with 3 columns
df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
5,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
6,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"
7,M8A\n,Not assigned\n,Not assigned\n
8,M9A\n,Etobicoke\n,"Islington Avenue, Humber Valley Village\n"
9,M1B\n,Scarborough\n,"Malvern, Rouge\n"


In [4]:
# Remove the unnecessary "\n" at the end of each string in the column "Neighbourhood" 

df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"
5,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights"
6,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government"
7,M8A\n,Not assigned\n,Not assigned
8,M9A\n,Etobicoke\n,"Islington Avenue, Humber Valley Village"
9,M1B\n,Scarborough\n,"Malvern, Rouge"


In [5]:
# Grouping the neighbourhoods along with their postal codes

df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn
5,M1H\n,Scarborough\n,Cedarbrae
6,M1J\n,Scarborough\n,Scarborough Village
7,M1K\n,Scarborough\n,"Kennedy Park, Ionview, East Birchmount Park"
8,M1L\n,Scarborough\n,"Golden Mile, Clairlea, Oakridge"
9,M1M\n,Scarborough\n,"Cliffside, Cliffcrest, Scarborough Village West"


In [6]:
# Print the size of the dataset

print("Shape: ", df.shape)

Shape:  (180, 3)


In [7]:
# Find the latitude and the longitude coordinates of the neighbourhoods

df_geo_coor = pd.read_csv("http://cocl.us/Geospatial_data")
df_geo_coor.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [8]:
# Compile 2 dataframes "df" and "df_geo_coor" into 1

df_London = pd.merge(df, df_geo_coor, how="left", left_on = "PostalCode", right_on = "Postal Code")
# remove the "Postal Code" column
df_London.drop("Postal Code", axis=1, inplace=True)
df_London.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A\n,Not assigned\n,Not assigned,NaN,NaN
1,M1B\n,Scarborough\n,"Malvern, Rouge",NaN,NaN
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek",NaN,NaN
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill",NaN,NaN
4,M1G\n,Scarborough\n,Woburn,NaN,NaN
5,M1H\n,Scarborough\n,Cedarbrae,NaN,NaN
6,M1J\n,Scarborough\n,Scarborough Village,NaN,NaN
7,M1K\n,Scarborough\n,"Kennedy Park, Ionview, East Birchmount Park",NaN,NaN
8,M1L\n,Scarborough\n,"Golden Mile, Clairlea, Oakridge",NaN,NaN
9,M1M\n,Scarborough\n,"Cliffside, Cliffcrest, Scarborough Village West",NaN,NaN


In [9]:
# Explore and cluster the neighbourhoods in London

# 1. step: Find the latitude and longitude values of London

address = "London, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geograpical coordinate of London is {}, {}." .format(latitude, longitude))

The geograpical coordinate of London is 42.9836747, -81.2496068.


In [10]:
# Import folium 

!pip install folium
import pandas as pd
import folium
print("Folium is imported.")

     |████████████████████████████████| 94 kB 5.7 MB/s  eta 0:00:01
Folium is imported.


In [11]:
# Get the map of London by using the above latitude and longitude values

map_London = folium.Map(location=[latitude, longitude], zoom_start=10)
map_London

In [12]:
CLIENT_ID = 'XNLTNFIBMD5UP2REJFZMTC5CFJ0UJWNV24RIIDA2QO41BWAN'
CLIENT_SECRET = 'O1LQAWJFTGIMOY2BCMNAIBQ4Y20ADJVZLYEHWPFRVGMLIJRL'
ACCESS_TOKEN = 'XMCQSPAIRH3GAALYYXK2PB2HGDSBOB15IKSJJGADUSOH4QCK'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XNLTNFIBMD5UP2REJFZMTC5CFJ0UJWNV24RIIDA2QO41BWAN
CLIENT_SECRET:O1LQAWJFTGIMOY2BCMNAIBQ4Y20ADJVZLYEHWPFRVGMLIJRL


In [13]:
address = 'Horton St E, London, ON'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)


42.9777906 -81.2514023


In [14]:
search_query = 'venues'
radius = 1500
print(search_query + ' are found!')

venues are found!


In [15]:
url = 'https://api.foursquare.com/v2/venues/search?client_id=XNLTNFIBMD5UP2REJFZMTC5CFJ0UJWNV24RIIDA2QO41BWAN&client_secret=O1LQAWJFTGIMOY2BCMNAIBQ4Y20ADJVZLYEHWPFRVGMLIJRL&ll=42.9777906,-81.2514023&oauth_token=XMCQSPAIRH3GAALYYXK2PB2HGDSBOB15IKSJJGADUSOH4QCK&v=20180604&query=restaurant&radius=1500&limit=30'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=XNLTNFIBMD5UP2REJFZMTC5CFJ0UJWNV24RIIDA2QO41BWAN&client_secret=O1LQAWJFTGIMOY2BCMNAIBQ4Y20ADJVZLYEHWPFRVGMLIJRL&ll=42.9777906,-81.2514023&oauth_token=XMCQSPAIRH3GAALYYXK2PB2HGDSBOB15IKSJJGADUSOH4QCK&v=20180604&query=restaurant&radius=1500&limit=30'

In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ff9e2f9dbed4b23977fcc5f'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4b97329ff964a520cbfb34e3',
    'name': 'Thaifoon Restaurant',
    'location': {'address': '120 Dundas St.',
     'crossStreet': 'near Talbot St.',
     'lat': 42.98331116257564,
     'lng': -81.25135712126716,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.98331116257564,
       'lng': -81.25135712126716}],
     'distance': 614,
     'postalCode': 'N6A 1G3',
     'cc': 'CA',
     'city': 'London',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['120 Dundas St. (near Talbot St.)',
      'London ON N6A 1G3',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d149941735',
      'name': 'Thai Restaurant',
      'pluralName': 'Thai Restaurants',
      'shortName': 'Thai',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/thai_',
       'suffix': '.png'

In [17]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head(10)


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4b97329ff964a520cbfb34e3,Thaifoon Restaurant,"[{'id': '4bf58dd8d48988d149941735', 'name': 'Thai Restaurant', 'pluralName': 'Thai Restaurants', 'shortName': 'Thai', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/thai_', 'suffix': '.png'}, 'primary': True}]",v-1610212089,False,120 Dundas St.,near Talbot St.,42.983311,-81.251357,"[{'label': 'display', 'lat': 42.98331116257564, 'lng': -81.25135712126716}]",614,N6A 1G3,CA,London,ON,Canada,"[120 Dundas St. (near Talbot St.), London ON N6A 1G3, Canada]",62339564
1,4bf532dd706e20a1e222aa98,Westside Family Restaurant,"[{'id': '4bf58dd8d48988d143941735', 'name': 'Breakfast Spot', 'pluralName': 'Breakfast Spots', 'shortName': 'Breakfast', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/breakfast_', 'suffix': '.png'}, 'primary': True}]",v-1610212089,False,107 Mount Pleasant Ave,Wharncliffe Rd. N.,42.984838,-81.263275,"[{'label': 'display', 'lat': 42.98483774629121, 'lng': -81.26327496848552}]",1245,N6H 1E1,CA,London,ON,Canada,"[107 Mount Pleasant Ave (Wharncliffe Rd. N.), London ON N6H 1E1, Canada]",NaN
2,4b61ce40f964a520fa232ae3,Tahinis Restaurant,"[{'id': '4bf58dd8d48988d115941735', 'name': 'Middle Eastern Restaurant', 'pluralName': 'Middle Eastern Restaurants', 'shortName': 'Middle Eastern', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/middleeastern_', 'suffix': '.png'}, 'primary': True}]",v-1610212089,False,11-551 Richmond St,at Albert St.,42.988600,-81.250732,"[{'label': 'display', 'lat': 42.9885998, 'lng': -81.2507319}]",1204,N6A 3E9,CA,London,ON,Canada,"[11-551 Richmond St (at Albert St.), London ON N6A 3E9, Canada]",561310659
3,4b5e63e5f964a520ba8c29e3,Symposium Cafe Restaurant & Lounge,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'Restaurant', 'pluralName': 'Restaurants', 'shortName': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}]",v-1610212089,False,620 Richmond St.,at Central Ave.,42.990655,-81.250828,"[{'label': 'display', 'lat': 42.990655149613, 'lng': -81.25082830194532}]",1432,N6J 5A9,CA,London,ON,Canada,"[620 Richmond St. (at Central Ave.), London ON N6J 5A9, Canada]",146445443
4,4e249a0d628469a5743552b0,Tanakaya Japanese Restaurant,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'Sushi Restaurant', 'pluralName': 'Sushi Restaurants', 'shortName': 'Sushi', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/sushi_', 'suffix': '.png'}, 'primary': True}]",v-1610212089,False,130 King St.,at Talbot St.,42.982631,-81.250504,"[{'label': 'display', 'lat': 42.98263111508381, 'lng': -81.25050388952862}]",543,N6A 1C5,CA,London,ON,Canada,"[130 King St. (at Talbot St.), London ON N6A 1C5, Canada]",NaN
5,4d44bb52e198721efd0ebf8b,Viet-thai Restaurant,"[{'id': '4bf58dd8d48988d149941735', 'name': 'Thai Restaurant', 'pluralName': 'Thai Restaurants', 'shortName': 'Thai', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/thai_', 'suffix': '.png'}, 'primary': True}]",v-1610212089,False,NaN,NaN,42.979698,-81.245114,"[{'label': 'display', 'lat': 42.979698, 'lng': -81.24511420726776}]",554,NaN,CA,NaN,NaN,Canada,[Canada],NaN
6,4bb73b62f562ef3b26223197,Family Circle Restaurant,"[{'id': '4bf58dd8d48988d143941735', 'name': 'Breakfast Spot', 'pluralName': 'Breakfast Spots', 'shortName': 'Breakfast', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/breakfast_', 'suffix': '.png'}, 'primary': True}]",v-1610212089,False,147 Wellington St,Grey st,42.977113,-81.241706,"[{'label': 'display', 'lat': 42.97711254709463, 'lng': -81.2417064808509}]",793,N6B 2K7,CA,London,ON,Canada,"[147 Wellington St (Grey st), London ON N6B 2K7, Canada]",NaN
7,4ee255259911340c36dae60b,Restaurant Equipmen

In [18]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
LondON_dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venues.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


LondON_dataframe_filtered['categories'] = LondON_dataframe_filtered.apply(get_category_type, axis=1)

LondON_dataframe_filtered.columns = [column.split('.')[-1] for column in LondON_dataframe_filtered.columns]

LondON_dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Thaifoon Restaurant,Thai Restaurant,120 Dundas St.,near Talbot St.,42.983311,-81.251357,"[{'label': 'display', 'lat': 42.98331116257564, 'lng': -81.25135712126716}]",614,N6A 1G3,CA,London,ON,Canada,"[120 Dundas St. (near Talbot St.), London ON N6A 1G3, Canada]",4b97329ff964a520cbfb34e3
1,Westside Family Restaurant,Breakfast Spot,107 Mount Pleasant Ave,Wharncliffe Rd. N.,42.984838,-81.263275,"[{'label': 'display', 'lat': 42.98483774629121, 'lng': -81.26327496848552}]",1245,N6H 1E1,CA,London,ON,Canada,"[107 Mount Pleasant Ave (Wharncliffe Rd. N.), London ON N6H 1E1, Canada]",4bf532dd706e20a1e222aa98
2,Tahinis Restaurant,Middle Eastern Restaurant,11-551 Richmond St,at Albert St.,42.988600,-81.250732,"[{'label': 'display', 'lat': 42.9885998, 'lng': -81.2507319}]",1204,N6A 3E9,CA,London,ON,Canada,"[11-551 Richmond St (at Albert St.), London ON N6A 3E9, Canada]",4b61ce40f964a520fa232ae3
3,Symposium Cafe Restaurant & Lounge,Restaurant,620 Richmond St.,at Central Ave.,42.990655,-81.250828,"[{'label': 'display', 'lat': 42.990655149613, 'lng': -81.25082830194532}]",1432,N6J 5A9,CA,London,ON,Canada,"[620 Richmond St. (at Central Ave.), London ON N6J 5A9, Canada]",4b5e63e5f964a520ba8c29e3
4,Tanakaya Japanese Restaurant,Sushi Restaurant,130 King St.,at Talbot St.,42.982631,-81.250504,"[{'label': 'display', 'lat': 42.98263111508381, 'lng': -81.25050388952862}]",543,N6A 1C5,CA,London,ON,Canada,"[130 King St. (at Talbot St.), London ON N6A 1C5, Canada]",4e249a0d628469a5743552b0
5,Viet-thai Restaurant,Thai Restaurant,NaN,NaN,42.979698,-81.245114,"[{'label': 'display', 'lat': 42.979698, 'lng': -81.24511420726776}]",554,NaN,CA,NaN,NaN,Canada,[Canada],4d44bb52e198721efd0ebf8b
6,Family Circle Restaurant,Breakfast Spot,147 Wellington St,Grey st,42.977113,-81.241706,"[{'label': 'display', 'lat': 42.97711254709463, 'lng': -81.2417064808509}]",793,N6B 2K7,CA,London,ON,Canada,"[147 Wellington St (Grey st), London ON N6B 2K7, Canada]",4bb73b62f562ef3b26223197
7,Restaurant Equipment Supply,Miscellaneous Shop,234 William St,NaN,42.983210,-81.231098,"[{'label': 'display', 'lat': 42.98321021475201, 'lng': -81.23109828903625}]",1760,NaN,CA,NaN,NaN,Canada,"[234 William St, Canada]",4ee255259911340c36dae60b
8,Five Fortune Culture restaurant,Chinese Restaurant,368 Richmond St,King St,42.982730,-81.248864,"[{'label': 'display', 'lat': 42.98273003770885, 'lng': -81.24886363809682}]",587,NaN,CA,London,ON,Canada,"[368 Richmond St (King St), London ON, Canada]",5324ef9d498e89a993a33014
9,Melody Restaurant,Diner,NaN,NaN,42.983746,-81.249484,"[{'label': 'display', 'lat': 42.983746, 'lng': -81.249484}]",681,NaN,CA,London,ON,Canada,"[London ON, Canada]",4e74e1fb8998ed82a43fcaf0


In [19]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
LondON_dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venues.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


LondON_dataframe_filtered.name

LondON_venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)
LondON_venues_map

In [20]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
LondON_dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venues.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


LondON_dataframe_filtered.name

LondON_venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='205 Horton St E',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(LondON_venues_map)

LondON_venues_map

In [21]:
address = 'Wonderland Rd, London, ON'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)


42.970244 -81.2922947


In [22]:
search_query = 'venues'
radius = 1500
print(search_query + ' are found!')

venues are found!


In [23]:
url = 'https://api.foursquare.com/v2/venues/search?client_id=XNLTNFIBMD5UP2REJFZMTC5CFJ0UJWNV24RIIDA2QO41BWAN&client_secret=O1LQAWJFTGIMOY2BCMNAIBQ4Y20ADJVZLYEHWPFRVGMLIJRL&ll=42.970244,-81.2922947&oauth_token=XMCQSPAIRH3GAALYYXK2PB2HGDSBOB15IKSJJGADUSOH4QCK&v=20180604&query=restaurant&radius=1500&limit=30'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=XNLTNFIBMD5UP2REJFZMTC5CFJ0UJWNV24RIIDA2QO41BWAN&client_secret=O1LQAWJFTGIMOY2BCMNAIBQ4Y20ADJVZLYEHWPFRVGMLIJRL&ll=42.970244,-81.2922947&oauth_token=XMCQSPAIRH3GAALYYXK2PB2HGDSBOB15IKSJJGADUSOH4QCK&v=20180604&query=restaurant&radius=1500&limit=30'

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ff9e443ccdb8b3199159357'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '514e0392e4b0d91e950ceeb6',
    'name': 'Spring Restaurant.',
    'location': {'lat': 42.97282,
     'lng': -81.286641,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.97282,
       'lng': -81.286641}],
     'distance': 542,
     'cc': 'CA',
     'country': 'Canada',
     'formattedAddress': ['Canada']},
    'categories': [{'id': '4bf58dd8d48988d1c0941735',
      'name': 'Mediterranean Restaurant',
      'pluralName': 'Mediterranean Restaurants',
      'shortName': 'Mediterranean',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mediterranean_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1610212419',
    'hasPerk': False},
   {'id': '5f45a23af568bf0ea95d063b',
    'name': 'Aleppo Restaurant',
    'location': {'address': '666 Wonderland Rd N',
     'c

In [25]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head(10)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.address,location.crossStreet,location.postalCode,location.neighborhood,location.city,location.state,venuePage.id
0,514e0392e4b0d91e950ceeb6,Spring Restaurant.,"[{'id': '4bf58dd8d48988d1c0941735', 'name': 'Mediterranean Restaurant', 'pluralName': 'Mediterranean Restaurants', 'shortName': 'Mediterranean', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mediterranean_', 'suffix': '.png'}, 'primary': True}]",v-1610212419,False,42.972820,-81.286641,"[{'label': 'display', 'lat': 42.97282, 'lng': -81.286641}]",542,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5f45a23af568bf0ea95d063b,Aleppo Restaurant,"[{'id': '5bae9231bedf3950379f89da', 'name': 'Syrian Restaurant', 'pluralName': 'Syrian Restaurants', 'shortName': 'Syrian Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/middleeastern_', 'suffix': '.png'}, 'primary': True}]",v-1610212419,False,42.984337,-81.293940,"[{'label': 'display', 'lat': 42.984337, 'lng': -81.29394}]",1574,CA,Canada,"[666 Wonderland Rd N (at Oxford St), London ON N6H 4K9, Canada]",666 Wonderland Rd N,at Oxford St,N6H 4K9,West London,London,ON,NaN
2,4c83cf7147cc224bb260929f,Riverdale Family Restaurant,"[{'id': '4bf58dd8d48988d143941735', 'name': 'Breakfast Spot', 'pluralName': 'Breakfast Spots', 'shortName': 'Breakfast', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/breakfast_', 'suffix': '.png'}, 'primary': True}]",v-1610212419,False,42.969146,-81.280536,"[{'label': 'display', 'lat': 42.969145777325096, 'lng': -81.2805358839232}]",965,CA,Canada,"[360 Springbank Dr, London ON, Canada]",360 Springbank Dr,NaN,NaN,NaN,London,ON,NaN
3,56d73a00cd1024d765d17abc,London Chinese Restaurant,"[{'id': '4bf58dd8d48988d1f5931735', 'name': 'Dim Sum Restaurant', 'pluralName': 'Dim Sum Restaurants', 'shortName': 'Dim Sum', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/dimsum_', 'suffix': '.png'}, 'primary': True}]",v-1610212419,False,42.983243,-81.290698,"[{'label': 'display', 'lat': 42.98324301573538, 'lng': -81.29069795926857}]",1452,CA,Canada,"[Unit 1A, 530 Oxford St West,, London ON N6H 1T6, Canada]","Unit 1A, 530 Oxford St West,",NaN,N6H 1T6,NaN,London,ON,NaN
4,4fde5babe4b0ede382ba7fde,Shanghi Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'Chinese Restaurant', 'pluralName': 'Chinese Restaurants', 'shortName': 'Chinese', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}]",v-1610212419,False,42.970746,-81.277920,"[{'label': 'display', 'lat': 42.970745811302685, 'lng': -81.27792049885842}]",1172,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4ec5cc1f0aaf9c3cca61c54a,The Springs Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'Restaurant', 'pluralName': 'Restaurants', 'shortName': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}]",v-1610212419,False,42.970741,-81.277911,"[{'label': 'display', 'lat': 42.970740925738085, 'lng': -81.27791101194173}]",1172,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,52f051bb498e0b24da455c94,Papa's Pizza & Restaurant,"[{'id': '4bf58dd8d48988d110941735', 'name': 'Italian Restaurant', 'pluralName': 'Italian Restaurants', 'shortName': 'Italian', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_', 'suffix': '.png'}, 'primary': True}]",v-1610212419,False,42.981899,-81.283638,"[{'label': 'display', 'lat': 42.98189926147461, 'lng': -81.28363800048828}]",1476,CA,Canada,"[117 St. George Street (Piccadilly Street), London ON, Canada]",117 St. George Street,Piccadilly Street,NaN,NaN,London,ON,NaN
7,4e31f4c2a809189f7a240013,Tehran Restaurant,"[{'id': '4bf58dd8d48988d115941735', 'name': 'Middle Eastern Restaurant', 'pluralName': 'Middle Eastern Restaurants', 'shortName': 'Midd

In [26]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
LondON_dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venues.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


LondON_dataframe_filtered['categories'] = LondON_dataframe_filtered.apply(get_category_type, axis=1)

LondON_dataframe_filtered.columns = [column.split('.')[-1] for column in LondON_dataframe_filtered.columns]

LondON_dataframe_filtered

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,address,crossStreet,postalCode,neighborhood,city,state,id
0,Spring Restaurant.,Mediterranean Restaurant,42.972820,-81.286641,"[{'label': 'display', 'lat': 42.97282, 'lng': -81.286641}]",542,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN,NaN,514e0392e4b0d91e950ceeb6
1,Aleppo Restaurant,Syrian Restaurant,42.984337,-81.293940,"[{'label': 'display', 'lat': 42.984337, 'lng': -81.29394}]",1574,CA,Canada,"[666 Wonderland Rd N (at Oxford St), London ON N6H 4K9, Canada]",666 Wonderland Rd N,at Oxford St,N6H 4K9,West London,London,ON,5f45a23af568bf0ea95d063b
2,Riverdale Family Restaurant,Breakfast Spot,42.969146,-81.280536,"[{'label': 'display', 'lat': 42.969145777325096, 'lng': -81.2805358839232}]",965,CA,Canada,"[360 Springbank Dr, London ON, Canada]",360 Springbank Dr,NaN,NaN,NaN,London,ON,4c83cf7147cc224bb260929f
3,London Chinese Restaurant,Dim Sum Restaurant,42.983243,-81.290698,"[{'label': 'display', 'lat': 42.98324301573538, 'lng': -81.29069795926857}]",1452,CA,Canada,"[Unit 1A, 530 Oxford St West,, London ON N6H 1T6, Canada]","Unit 1A, 530 Oxford St West,",NaN,N6H 1T6,NaN,London,ON,56d73a00cd1024d765d17abc
4,Shanghi Chinese Restaurant,Chinese Restaurant,42.970746,-81.277920,"[{'label': 'display', 'lat': 42.970745811302685, 'lng': -81.27792049885842}]",1172,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN,NaN,4fde5babe4b0ede382ba7fde
5,The Springs Restaurant,Restaurant,42.970741,-81.277911,"[{'label': 'display', 'lat': 42.970740925738085, 'lng': -81.27791101194173}]",1172,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN,NaN,4ec5cc1f0aaf9c3cca61c54a
6,Papa's Pizza & Restaurant,Italian Restaurant,42.981899,-81.283638,"[{'label': 'display', 'lat': 42.98189926147461, 'lng': -81.28363800048828}]",1476,CA,Canada,"[117 St. George Street (Piccadilly Street), London ON, Canada]",117 St. George Street,Piccadilly Street,NaN,NaN,London,ON,52f051bb498e0b24da455c94
7,Tehran Restaurant,Middle Eastern Restaurant,42.984683,-81.289118,"[{'label': 'display', 'lat': 42.98468290152073, 'lng': -81.28911769007844}]",1628,CA,Canada,"[500 Oxford St. W (Proudfoot), London ON, Canada]",500 Oxford St. W,Proudfoot,NaN,NaN,London,ON,4e31f4c2a809189f7a240013
8,McGinnis Restaurant and Bar,Gastropub,42.984885,-81.294019,"[{'label': 'display', 'lat': 42.98488458797959, 'lng': -81.29401861284907}]",1635,CA,Canada,"[666 Wonderland Rd. N., London ON N6H 4K9, Canada]",666 Wonderland Rd. N.,NaN,N6H 4K9,NaN,London,ON,4b50d43bf964a520083427e3
9,Getaway Restaurant,None,42.986754,-81.295429,"[{'label': 'display', 'lat': 42.986754, 'lng': -81.295429}]",1855,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN,NaN,4d1f7917b69c6dcb70836795


In [27]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
LondON_dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venues.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


LondON_dataframe_filtered.name

LondON_venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)
LondON_venues_map

In [29]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
LondON_dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venues.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


LondON_dataframe_filtered.name

LondON_venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='710 Wonderland Rd',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(LondON_venues_map)

LondON_venues_map

In [30]:
address = 'Commissioners Rd, London, ON'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

42.9598834 -81.1888812


In [31]:
search_query = 'venues'
radius = 1500
print(search_query + ' are found!')

venues are found!


In [32]:
url = 'https://api.foursquare.com/v2/venues/search?client_id=XNLTNFIBMD5UP2REJFZMTC5CFJ0UJWNV24RIIDA2QO41BWAN&client_secret=O1LQAWJFTGIMOY2BCMNAIBQ4Y20ADJVZLYEHWPFRVGMLIJRL&ll=42.9598834,-81.1888812&oauth_token=XMCQSPAIRH3GAALYYXK2PB2HGDSBOB15IKSJJGADUSOH4QCK&v=20180604&query=restaurant&radius=1500&limit=30'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=XNLTNFIBMD5UP2REJFZMTC5CFJ0UJWNV24RIIDA2QO41BWAN&client_secret=O1LQAWJFTGIMOY2BCMNAIBQ4Y20ADJVZLYEHWPFRVGMLIJRL&ll=42.9598834,-81.1888812&oauth_token=XMCQSPAIRH3GAALYYXK2PB2HGDSBOB15IKSJJGADUSOH4QCK&v=20180604&query=restaurant&radius=1500&limit=30'

In [33]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ff9e5d955e19a6657b36e27'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4be732662468c928e5380243',
    'name': 'Fireside Family Restaurant',
    'location': {'lat': 42.9624414613352,
     'lng': -81.19734102865297,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.9624414613352,
       'lng': -81.19734102865297}],
     'distance': 745,
     'cc': 'CA',
     'country': 'Canada',
     'formattedAddress': ['Canada']},
    'categories': [{'id': '4bf58dd8d48988d14e941735',
      'name': 'American Restaurant',
      'pluralName': 'American Restaurants',
      'shortName': 'American',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1610212825',
    'hasPerk': False}]}}

In [34]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head(10)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress
0,4be732662468c928e5380243,Fireside Family Restaurant,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'American Restaurant', 'pluralName': 'American Restaurants', 'shortName': 'American', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}]",v-1610212825,False,42.962441,-81.197341,"[{'label': 'display', 'lat': 42.9624414613352, 'lng': -81.19734102865297}]",745,CA,Canada,[Canada]


In [35]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
LondON_dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venues.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


LondON_dataframe_filtered['categories'] = LondON_dataframe_filtered.apply(get_category_type, axis=1)

LondON_dataframe_filtered.columns = [column.split('.')[-1] for column in LondON_dataframe_filtered.columns]

LondON_dataframe_filtered

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,id
0,Fireside Family Restaurant,American Restaurant,42.962441,-81.197341,"[{'label': 'display', 'lat': 42.9624414613352, 'lng': -81.19734102865297}]",745,CA,Canada,[Canada],4be732662468c928e5380243


In [36]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
LondON_dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venues.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


LondON_dataframe_filtered.name

LondON_venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)
LondON_venues_map

In [37]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
LondON_dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venues.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


LondON_dataframe_filtered.name

LondON_venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='800 Commissioners Rd',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(LondON_venues_map)

LondON_venues_map